<a href="https://colab.research.google.com/github/EliasSf73/test_/blob/master/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

basic libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets,transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

Convert MNIST  dataset in to 4D-Tensor: # of images, heights,widths,color channel

In [2]:
transform=transforms.ToTensor()

TRAIN DATA

In [3]:
train_data=datasets.MNIST(root='/CNN_dataset',train=True,download=True,transform=transform)

100%|██████████| 9912422/9912422 [00:00<00:00, 276601608.98it/s]

Extracting /CNN_dataset/MNIST/raw/train-images-idx3-ubyte.gz to /CNN_dataset/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 77601341.34it/s]


Extracting /CNN_dataset/MNIST/raw/train-labels-idx1-ubyte.gz to /CNN_dataset/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 172690056.85it/s]

Extracting /CNN_dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to /CNN_dataset/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 3316018.93it/s]


Extracting /CNN_dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to /CNN_dataset/MNIST/raw



TEST DATA

In [4]:
test_data=datasets.MNIST('/CNN_dataset',train=False,download=True,transform=transform)

In [5]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /CNN_dataset
    Split: Train
    StandardTransform
Transform: ToTensor()

In [6]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /CNN_dataset
    Split: Test
    StandardTransform
Transform: ToTensor()

Create a batch size for the images: 10 for this project

In [7]:
train_loader=DataLoader(train_data,batch_size=10,shuffle=True)
test_loader=DataLoader(test_data,batch_size=10,shuffle=False)

THE MODEL

In [16]:
class ConvNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    # locally connected convolutional layers
    self.conv1=nn.Conv2d(1,6,3,1)  # num_inputs,num_outputs,kernel size, step/strides taken by kernel
    self.conv2=nn.Conv2d(6,16,3,1)
    # fully connected layers from ordinary neural network
    self.layer1=nn.Linear(5*5*16,120)
    self.layer2=nn.Linear(120,84)
    self.layer3=nn.Linear(84,10) # 10 is number of classes

  def forward(self,x):
    #1st pass
    x=F.relu(self.conv1(x))
    x=F.max_pool2d(x,2,2) # 2*2 kernel for pooling with 2 strides
    # second pass
    x=F.relu(self.conv2(x))
    x=F.max_pool2d(x,2,2)
    # change 'view' of x to flatten it out before passing to 'layer1'
    x=x.view(-1,16*5*5)
    # pass to fully_connected layer
    x=F.relu(self.layer1(x))
    x=F.relu(self.layer2(x))
    x=F.relu(self.layer3(x))
    # get modified x back
    return F.log_softmax(x,dim=1)

In [26]:
torch.manual_seed(42)
model=ConvNetwork()
model

ConvNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (layer1): Linear(in_features=400, out_features=120, bias=True)
  (layer2): Linear(in_features=120, out_features=84, bias=True)
  (layer3): Linear(in_features=84, out_features=10, bias=True)
)

OPTIMIZATION

In [27]:
loss_module=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)

TRAIN AND EVALUATE

In [ ]:
epochs=5
for epoch in range(epochs):
  train_cr=0
  test_cr=0
  for i,(X_train,y_train) in enumerate(train_loader):
    i+=1     #start our batches at 1
    y_pred=model(X_train)  # predict values
    loss=loss_module(y_pred,y_train)      # compute associated losses: how off is the prediction?


    # update parameters
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # print some results
    if i%500==0:
      print(f'epoch: {epoch}, batch: {i}, loss: {loss.item()}')


  # Test on test dataset
  with torch.no_grad():
    for j,(X_test,y_test) in enumerate(test_loader):
      y_val=model(X_test)
      loss=loss_module(y_val,y_test)